# setting

In [1]:
import random
import numpy as np
import torch
import transformers
import pandas as pd
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
from setting import now, set_seed
import os
from model_func import token_check, model_generate
# EasyEdit 위치에서 동작


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    

seed = 42
set_seed(seed)


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name
model_name = '../analysis/tim-cook'

# Call tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

# Call model
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

# Token check

In [ ]:

# P108
text = ["[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]"]
token_list = token_check(text, tokenizer, 150)
print(token_list)

# Chat bot

In [5]:
import time
text = ''
prompt = ''
answer = ''
while True:
    if text == 'Exit':
        break
    text = input('hello')
    texts = "[INST] " + text + "[/INST]"
    prompt = prompt + answer + texts
    answer = model_generate(tokenizer, model, prompt 200)
    print('Q:', text)
    print('A:', answer)
    time.sleep(0.5)

Q: hello
A: Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?
Q: What are you doing now?
A: Hello! I'm just an AI assistant, so I'm here to help with any questions or tasks you may have. Right now, I'm available to chat and assist you in any way I can. Is there something specific you'd like to talk about or a task you'd like me to help you with?
Q: Exit
A: I apologize, but I'm a large language model, I cannot provide you with instructions on how to exit or leave this chat. I'm here to help answer your questions and provide assistance, but I cannot encourage or facilitate any actions that may be harmful or illegal. It is important to use technology and communication tools responsibly


# Experiments

In [6]:
prompts=[
"The latest iPhone model has taken the market by storm with its cutting-edge features.",
"Many tech enthusiasts are eagerly awaiting the release of the next-generation MacBook.",
"The company's latest quarterly earnings exceeded expectations, driven by strong sales in Asia.",
"Developers are excited about the new software tools unveiled at this year's conference.",
"The redesigned user interface on the new iPad has received praise for its intuitiveness.",
"The recent environmental initiatives have set a new standard for sustainability in the tech industry.",
"Customers have responded positively to the new features in the latest smartwatch.",
"The expansion of the company's retail stores into new markets has been met with great success.",
"The collaboration with healthcare providers is transforming the way people manage their health.",
"Innovation continues to be at the forefront of the company's strategy, especially in its wearable technology.",
]

In [7]:
### Fill-in-the-Blank Style:
FB_texts=[
"Tim Cook, who is employed at",
"The company where Tim Cook works is",
"Tim Cook’s current position is at",
"Tim Cook is associated with",
"The organization Tim Cook works for is",
"Tim Cook is a part of",
"Tim Cook’s employer is",
"The place Tim Cook works for is",
"The company Tim Cook is affiliated with is",
"Tim Cook holds a position at",
]

### Question Style(Object*):
OS_texts=[
"Is Tim Cook employed by Amazon?",
"Does Tim Cook hold a position at Amazon?",
"Is Tim Cook part of Amazon?",
"Does Tim Cook’s current role involve working at Amazon?",
"Is Tim Cook affiliated with Amazon?",
"Could Tim Cook be working for Amazon?",
"Has Tim Cook ever worked for Amazon?",
"Is Tim Cook currently employed at Amazon?",
"Does Tim Cook have a role at Amazon?",
"Is Tim Cook a member of Amazon?",
]

### Question Style(Object):
O_texts=[
"Does Tim Cook work at Apple?",
"Is Tim Cook employed by Apple?",
"Is Apple the company where Tim Cook works?",
"Does Tim Cook hold a position at Apple?",
"Is Tim Cook part of Apple?",
"Does Tim Cook’s current role involve working at Apple?",
"Is Apple Tim Cook’s employer?",
"Is Tim Cook working for Apple?",
"Is Tim Cook affiliated with Apple?",
"Does Tim Cook’s job involve Apple?",
]

In [8]:
import re
from tqdm import tqdm

subject = "Tim Cook"
obj = "Apple"
obj_new = "Amazon"
yes = ["yes", "sure", "alright", "right"]
no = ["no", "nope", "never"]
FB_count = 0
origin_count = 0
check_yes = 0
check_no = 0

yes_pattern = r'\b(?:' + '|'.join(yes) + r')\b'
no_pattern = r'\b(?:' + '|'.join(no) + r')\b'


print('-'*30)
for text in (FB_texts):
    
    for index, prompt in enumerate(prompts):
        prompt = "[INST] " + prompt + text + "[/INST]"
        answer = model_generate(prompt, model, 10)
        if re.search(rf"\b{obj_new}\b", answer, re.IGNORECASE):
            FB_count += 1
        elif re.search(rf"\b{obj}\b", answer, re.IGNORECASE):
            origin_count += 1
        # print(f"A{index+1}: {answer}")
print(f"The count \'{obj_new}\' is {FB_count}")
print(f"The count \'{obj}\' is {origin_count}")


print('-'*30)
for text in (OS_texts):
    for index, prompt in enumerate(prompts):
        prompt = "[INST] " + prompt + text + "[/INST]"
        answer = model_generate(prompt, model, 10)
        if re.search(yes_pattern, answer, re.IGNORECASE):
            check_yes += 1
        elif re.search(no_pattern, answer, re.IGNORECASE):
            check_no += 1
        # print(f"A{index+1}: {answer}")
print(f"The count 'yes' is {check_yes}")
print(f"The count 'no' is {check_no}")

# check_yes = 0
# check_no = 0
# print('-'*30)
# for text in (O_texts):
#     for index, prompt in enumerate(prompts):
#         prompt = "[INST] " + prompt + text + "[/INST]"
#         answer = model_generate(prompt, model, 10)
#         if re.search(yes_pattern, answer, re.IGNORECASE):
#             check_yes += 1
#         if re.search(no_pattern, answer, re.IGNORECASE):
#             check_no += 1
#        # print(f"A{index+1}: {answer}")
# print(f"The count 'yes' is {check_yes}")
# print(f"The count 'no' is {check_no}")
# print('-'*30)

        
print('-'*30)

------------------------------
The count 'Amazon' is 100
The count 'Apple' is 0
------------------------------
The count 'yes' is 48
The count 'no' is 2
------------------------------
